# 1. File Loading

In [43]:
import torch
import pandas as pd
import numpy as np
import os

# load data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# train
from torch import nn
from torch.nn import functional as F
import torch.nn.init

# visualization
import matplotlib.pyplot as plt


In [44]:
batch_size = 32
num_workers = 40
train_set = './data/slim_train_set1.csv'
test_set = './data/slim_test_set1.csv'

In [45]:
class MFCCDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file):
        self.raw = pd.read_csv(csv_file, header=None, sep=r'\s*,\s*')
        self.label = torch.IntTensor(np.array(self.raw[0].values).reshape(len(self.raw[0].values), 1))
        self.len = len(self.label)
        self.data = torch.Tensor(np.array(self.raw.loc[:,1:]).reshape(len(self.label),100,500))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

print("Loading data...")

dataset = MFCCDataset(train_set)

dataloader = torch.utils.data.DataLoader(
   dataset, batch_size = batch_size, num_workers = num_workers, drop_last=True
)
print("data is ready!")

Loading data...
data is ready!


In [46]:
len(dataset.__getitem__(1921)[0][0])

500

# 2. Neural Network

In [47]:
import random
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용\
print("다음 기기로 학습합니다:", device)
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

다음 기기로 학습합니다: cuda


In [48]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1) -> 1, 100, 1000, 1
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=13, stride=1, padding=6),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=10, stride=10))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=7, stride=1, padding=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=5))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(1 * 5 * 128, 3, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [49]:
# CNN 모델 정의
learning_rate = 0.001
training_epochs = 25
model = CNN()   #.to(device)
model = torch.nn.DataParallel(model)
model.cuda()
criterion = torch.nn.CrossEntropyLoss() #.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_batch = len(dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 60


In [50]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.reshape(32,1,100,500).to(device)
        Y = Y.reshape(32,1)[:,0].to(device, dtype=torch.int64)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.73523891
[Epoch:    2] cost = 0.521369576
[Epoch:    3] cost = 0.344502747
[Epoch:    4] cost = 0.312511742
[Epoch:    5] cost = 0.283719301
[Epoch:    6] cost = 0.24042353
[Epoch:    7] cost = 0.223650903
[Epoch:    8] cost = 0.209939048
[Epoch:    9] cost = 0.189004794
[Epoch:   10] cost = 0.177645206
[Epoch:   11] cost = 0.168754488
[Epoch:   12] cost = 0.164400622
[Epoch:   13] cost = 0.130084425
[Epoch:   14] cost = 0.152773693
[Epoch:   15] cost = 0.153566509
[Epoch:   16] cost = 0.176035985
[Epoch:   17] cost = 0.122159734
[Epoch:   18] cost = 0.0939705223
[Epoch:   19] cost = 0.079269588
[Epoch:   20] cost = 0.0698852688
[Epoch:   21] cost = 0.0834453255
[Epoch:   22] cost = 0.0865378007
[Epoch:   23] cost = 0.0819080025
[Epoch:   24] cost = 0.0739739612
[Epoch:   25] cost = 0.0534079261


In [51]:
print("Loading testset...")

testset = MFCCDataset(test_set)

dataloader = torch.utils.data.DataLoader(
   testset, batch_size = batch_size, num_workers = num_workers, drop_last=True
)
print("testset is ready!")

Loading testset...
testset is ready!


In [52]:
import time
start = time.time()
with torch.no_grad():
    X_test = testset.data.view(len(testset), 1, 100, 500).float().to(device)
    Y_test = testset.label.to(device, dtype=torch.int64)[:,0]

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
print("time :", time.time() - start)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/ubuntu/Workspace/Kichang/KC/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/ubuntu/Workspace/Kichang/KC/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "<ipython-input-48-63f12f68c121>", line 34, in forward
    out = self.layer1(x)
  File "/home/ubuntu/Workspace/Kichang/KC/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/ubuntu/Workspace/Kichang/KC/lib/python3.8/site-packages/torch/nn/modules/container.py", line 117, in forward
    input = module(input)
  File "/home/ubuntu/Workspace/Kichang/KC/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **k